# Now just using DF directly
## no longer need to parse separately into lists etc.  all done internally
## TO Do the dump load and initialize the DB, need to make sure correct order pf tables is loaded first those with no dependencies and then increasingly so.
## now only erroring on original 3 .txt noted by Ian Gorton in original commnication
- - 1. Conf
- - 2. Pub , Keys, Authors, Affiliations (use sets to create unique ones but may change to all .unique)
- - 3. Composites (Many to Many)
- - 4. Paper/abstracts



In [49]:
import sqlite3
import pandas as pd

In [71]:
import sqlcommands as cmd
cmd = reload(cmd)

(1548, 7)


In [51]:
df = cmd.createTOTALTable()
df.iloc[50:60]

,Abstract,Author affiliation,Authors,Conf,Title,terms,year
50,The past 20 years has seen significant investm...,"IBM Software Group, Raleigh, NC, United States...","['Brown, Alan W', '', '', ' McDermid, John A',...",ECSA,The art and science of software architecture,"['Failure analysis', 'Systems engineering', 'O...",2007
51,"Nowadays, Wireless Sensor Networks WSN are a...",Divisi n de Sistemas e Ingenier a Electr nica ...,"['Losilla, Fernando', '', '', 'Vicente Chicote...",ECSA,Wireless sensor network application developmen...,"['Mathematical models', 'Software architecture...",2007
52,Automated services help enterprises create new...,"INFOLAB, Tilburg University, Dept of Informat...","['Papazoglou, Michael P', '', '']",ECSA,What s in a service,"['Condition monitoring', 'Electronic commerce'...",2007
53,The term co evolution describes the symbiotic ...,"University of St Andrews, St Andrews, KY16 9SX...","['Morrison, Ron', '', '', 'Balasubramaniam, Dh...",ECSA,An active architecture approach to dynamic sys...,"['Dynamical systems', 'Grid computing', 'Senso...",2007
54,Software architecture views represent the basi...,Department of Information Systems and Computat...,"['Cordero, Rogelio Limon', '', '', 'Salavert, ...",ECSA,Analyzing styles of the modular software archi...,"['Computer aided design', 'Requirements engine...",2007
55,"Currently, most software systems have a dynami...",Department of Information Systems and Computat...,"['Costa, Crist bal', '', '', 'Ali, Nour', '', ...",ECSA,Dynamic reconfiguration of software architectu...,"['Computer aided design', 'Object oriented pro...",2007
56,We propose a pattern based software developmen...,Department of Computational and Cognitive Scie...,"['C t , Isabelle', '', '', 'Heisel, Marina', '...",ECSA,Pattern based evolution of software architectures,"['Pattern recognition', 'Requirements engineer...",2007
57,Most of the research in the area of multimodal...,"Department of Computer Science, Technische Uni...","['Pereira, Alessandro Costa', ' ,', '', ' Hart...",ECSA,A distributed staged architecture for multimod...,"['Computer programming languages', 'Distribute...",2007
58,The vast diversity of implementation and suppo...,"Computer Languages and Systems II Department, ...","['L pez Sanz, Marcos', '', '', 'Acu a, C sar J...",ECSA,UML profile for the platform independent model...,"['Computer simulation', 'Computer software por...",2007
59,No abstract available,None,"['Dauvin, Jean Claude', ' Romana, Louis Alexan...",ECSA,Hydrobiologia Preface,[nan],2007


In [ ]:
#don't need 
#df1 = cmd.getPatentDataFrame(hdffile='../DataBaseParsing/DFstore4.h5') 

In [52]:
df.iloc[0]['Author affiliation']

u'DISP, Univ  of Rome, Rome, Italy'

# Create the Conf Table

In [53]:
confDF = cmd.createConfTable(df)

Opened Abstracts_DB.db database successfully
Created DataFrame
Records created successfully


In [54]:
confDF.head()

,confID,confName
0,1,ECSA
1,2,QoSA
2,3,WICSA


# Create publications Table
## Now can parse all the orginal .txt with exception of the same 3  Ian gorton noted were errant

In [55]:
pubDF = cmd.createPublicationsTable(df)

Opened Abstracts_DB.db database successfully
table dropped
Created PUBLICATIONS table
Created DataFrame
Records created successfully


In [56]:
pubDF.head()
#pubDF.to_csv('Tables_v1/publications.csv', sheet_name = 'publications')

,pubID,year,confName
0,1,2007,ECSA
1,2,2008,ECSA
2,3,2009,ECSA
3,4,2010,ECSA
4,5,2011,ECSA


# Create Keys Table

## There are errors here and I have spent way to ong trying to figure if out.  
## Somewhere the abstracts are being split into the terms.

In [57]:
keys = cmd.createKEYSTable(df)

Opened Abstracts_DB.db database successfully
              keyword
0     Loose couplings
1   aspectual concept
2       Discrete time
3  Polyacrylonitriles
4     Tacit knowledge
Created DataFrame
Records created successfully


In [ ]:
print len(keys)
print len(keys.keyword.unique())
keys.iloc[50:75]
keys.head(20)

# Authors
## I don't know how to parse them better!  Each text file seems to look at them differently and no idea what to do

In [58]:
authorsDF = cmd.createAUTHORSTable(df)

Opened Abstracts_DB.db database successfully
Author set created
Records created successfully


In [59]:
print len(authorsDF.authorName.drop_duplicates())  #<<--- double check!  will keep one 'empty ' ' becuase of set)
authorsDF.head()

2911


,authorID,authorName
0,1,Kolb Ronny
1,2,Von Massow Robert
2,3,Smiley Karen
3,4,Prakash Jai
4,5,Zhang Hongye


In [ ]:
DEFAULTDB = mydb
def createAUTHORSTable(data_frame, 
                    db = DEFAULTDB, 
                    DFCol = 'Authors',
                    table = 'AUTHORS' ,
                    tableCol = ['authorName']
                   ):
    ''' Creating the AUTHORS Table with 
            authorID as AUTOINCREMENTED PRIMARY KEY, 
             : param data_frame : Pandas DataFrame from which to create the AUTHORS TABLE
             : param db : str. Name of db. (ie. 'Abstracts.db')
                 : default 'test.db'
             : param DFCol : tuple of strings Column names of data_frame with groupby cols.
                 : default = 'Authors'
                 : alternatively : enter with already unique table.
             : param table : str. Name of Table to create or insert.
                 : default : AUTHORS
             : param tableCols : tuple of str. Table keywords as tuple of strings
                 : default  : ('authorName')
             : **NEED TO CREATE COMPOSITE **
         
             : output : Pandas DataFrame as output for inspection
             : output : sql table created with rows inserted.
         '''
    
    def findAuthorsSet(data_frame, DFCol = DFCol):
        ''' Parse the TOTALABSTRACTS table dataframe terms column.
        Create a single set of all terms
        
        : param data_frame : a Pandas DataFrame (ie TOTALABSTRACTS table as DF)
        : param DFCol : column name to be recast and set of terms found from
        
        : output : a master set of terms set as a list (no duplicates)
        '''
        import re
        commaQuote = re.compile('[\',]')
        
    
        #recast - since made a string during the entry into the sqlite db.
        authors = data_frame[DFCol].apply(lambda a: a.strip("[]'"))
        authors = authors.apply(lambda x: filter(lambda a: a != '',
                                                  commaQuote.sub(' ', x).split('   ')))
        
        
        #create single set and remove leading white space
        authors = list(frozenset().union(*authors))
        authors = [anew.strip(' ') for anew in authors]
        #need to reset becuase of the white space
        return list(set(authors))
    with sqlite3.connect(db) as con:
        print ("Opened %s database successfully" %db); 
        cur = con.cursor() 
        #drops the Authors table if it exist in db already
        cur.execute("DROP TABLE IF EXISTS " + table)
        #and recreates it
        cur.execute("CREATE TABLE " + table + "(\
        authorID INTEGER PRIMARY KEY AUTOINCREMENT, \
        authorName TEXT \
        )") 
        authors = findAuthorsSet(data_frame)
        print ('Author set created')
        
        #only take 1 -> becuase first is empty
        authorsDF = pd.DataFrame(authors[1:], columns = tableCol)
        
        #insert into the table
        authorsDF.to_sql(table, con, flavor='sqlite', 
                    schema=None, if_exists='append', 
                    index=False, index_label=None,
                    chunksize=None, dtype=None)
        print("Records created successfully");
        
    sql = "SELECT * FROM " + table
    dfAuthors = pd.read_sql_query(sql, con)
        
    #return table as Pandas DataFrame for inspection
    return dfAuthors

In [60]:
authorsDF = cmd.createAUTHORSTable(df)
authorsDF.head()

Opened Abstracts_DB.db database successfully
Author set created
Records created successfully


,authorID,authorName
0,1,Kolb Ronny
1,2,Von Massow Robert
2,3,Smiley Karen
3,4,Prakash Jai
4,5,Zhang Hongye


# Affiliation Table
## simple no extra parsing do to differences between entries

In [61]:
affilDF= cmd.createAFFILIATIONTable(df)

Opened Abstracts_DB.db database successfully
Created DataFrame
Records created successfully


In [62]:
affilDF.head()
#affilDF.to_csv('Tables_v1/affiliations.csv', sheet_name = 'affilations')

,affilID,affiliation
0,1,"DISP, Univ of Rome, Rome, Italy"
1,2,"LSTS ENIT, Tunis, Tunisia"
2,3,"INRIA, Univ des Sci et Technol de Lille, Vi..."
3,4,"Sch of Comput Sci , Univ of Adelaide, Adela..."
4,5,"Software Group, IBM, Raleigh, NC, United States"


# PAPER TABLE
## should this actually be made by first doing a join or something with the pub table?

In [63]:
paperDF= cmd.createPAPERTable(df)

Opened Abstracts_DB.db database successfully
table dropped
Created PAPER table
Created DataFrame
Records created successfully


In [64]:
paperDF.iloc[0].terms

u"['Software design', 'Structural properties', 'Feature models', 'Product-lines', 'Research challenges', 'Software product line architecture', 'Software product lines', 'Software systems', 'Structural Design', 'Computer Programming', 'Materials Science']"

In [ ]:
mydb = 'Abstracts_DB.db'
#paperDF.to_csv('Tables_v1/paper.csv', sheet_name = 'paper')
paperDFbyYrandConf = paperDF[['pubYear', 'confName', 'paperID']].groupby(['pubYear', 'confName'], axis = 0)
#paperDFbyYrandConf.count().to_html()
paperDFbyYrandConf.get_group((2007, 'ECSA'))

In [ ]:
def getPapersConfYr(yr, conf):
    with sqlite3.connect(mydb) as con:
        sqlcmd = "SELECT pubYear, confName, paperID, title, abstract FROM PAPER"
        PAPdf = pd.read_sql_query(sqlcmd, con)
    
        group = PAPdf.groupby(['pubYear', 'confName'], axis = 0)
        subgrp = group.get_group((yr, conf))
        
        return subgrp

In [ ]:
y = getPapersConfYr(2004, 'WICSA')
y.index

In [ ]:
mytable = []
for idx in y.index.get_values():
    entry = {}
    entry['paperID'] = y.loc[idx]['paperID']
    entry['title'] = y.loc[idx]['title']
    entry['abstract'] = y.loc[idx]['abstract']
    mytable.append(entry)
x = dict(data = mytable)
    

In [ ]:
x

# PAPERKEY COMPOSITE

In [65]:
pk = cmd.createPAPERKEYTable(paperDF)

sqlcommands.py:556: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  parsedPaper['NEWTERMS'] = parsedPaper[DFCol[-1]].apply(lambda x:


Opened Abstracts_DB.db database successfully
Records created successfully


In [66]:
pk.head()

,paperID,keyword
0,1,Research challenges
1,1,Software product line architecture
2,1,Materials Science
3,1,Structural properties
4,1,Software product lines


In [ ]:
pk.tail()

In [ ]:
paperKey = pk.merge(paperDF[['paperID', 'title','confName', 'pubYear']], on = 'paperID')
paperKey.head()

In [ ]:
def getPapersKWgroup(grouper):
    with sqlite3.connect('Abstracts_DB.db') as con:
        sqlcmd = "SELECT paperID, title, confName, pubYear FROM PAPER "
        
        paperdf = pd.read_sql_query(sqlcmd, con)
        
        sqlcmd2 = "SELECT paperID, keyword FROM PAPERKEY "
        kwdf = pd.read_sql_query(sqlcmd2, con)
        kwdf['keyword'] = kwdf['keyword'].apply(lambda word: eval(word))
        
        merged = kwdf.merge(paperdf, on = 'paperID')
        
        subgrp = merged.groupby(grouper)
        
        return merged, subgrp

In [ ]:
def getPapersKWgroup(grouper):
    with sqlite3.connect('Abstracts_DB.db') as con:
        sqlcmd = "SELECT paperID, title, confName, pubYear FROM PAPER "
        
        paperdf = pd.read_sql_query(sqlcmd, con)
        
        sqlcmd2 = "SELECT paperID, keyword FROM PAPERKEY "
        kwdf = pd.read_sql_query(sqlcmd2, con)
        kwdf['keyword'] = kwdf['keyword'].apply(lambda word: eval(word))
        
        merged = kwdf.merge(paperdf, on = 'paperID')
        
        subgrp = merged.groupby(grouper)
        
        return merged, subgrp

def getKWTrend(keyword):
    paperFreq = getPapersKWgroup('keyword')
    
    #paperFreq['keyword'] = paperFreq['keyword'].apply(lambda word: eval(word))
    
    return paperFreq


In [ ]:
paperKeyword = paperKey.groupby('paperID').count()
paperKwYr = paperKey.groupby('pubYear')

In [ ]:
paperKwYr.get_group(2010)[['paperID', 'keyword']].groupby('keyword').count().sort('paperID', ascending = False)  

In [ ]:
#enter a keyword and see trend over conference and year.
try1 = paperKey
try1 = try1.groupby('keyword')

In [ ]:
paperFreq, merged = getPapersKWgroup('keyword')

In [ ]:
overallbreakdown = paperFreq.count()
overallbreakdown.reset_index(inplace = True)
overallbreakdown.head()

In [ ]:
#import matplotlib as plt
%matplotlib inline
def confYrKeywords(top = 10):
    grouper = ['confName', 'pubYear']
    m, f = getPapersKWgroup(grouper)
    myentries = []
    for group in f.groups.keys():
        print group
        keywordcts = f.get_group((group)).groupby(["keyword"])["keyword"].count()
        
        kwdftop = keywordcts.sort_values(ascending = False).head(top)
        
        resetKW = pd.DataFrame(kwdftop).rename(columns = {'keyword' : 'count'})
        entry = {}
        entry['Group'] = group
        
        entry['Pie'] = getPieOne(resetKW, group)
        entry['Counts'] = resetKW.to_html()
        
        myentries.append(entry)
    
    return dict(data = myentries)
        
        
        

In [ ]:

keywordcts = f.get_group(('ECSA', 2007)).groupby(["keyword"])["keyword"].count()

top10 = keywordcts.sort_values(ascending = False)
top10.head()

In [ ]:

keywordcts.head(10).plot(kind = 'pie')
getPieOne(keywordcts.head(10), ('ECSA', 2007) )
kwdf = keywordcts.head(10)
new = pd.DataFrame(kwdf).rename(columns = {'keyword' : 'count'})
new
getPieOne(new, ('ECSA', 2007))
getBar(new, ('ECSA', 2007))

In [ ]:
tryit = confYrKeywords()



In [ ]:
tryit

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cStringIO import StringIO
import base64
import seaborn as sns
%matplotlib inline

def getPieOne(df, conference):
    fig = plt.figure()
    fig = df.plot(kind = 'pie', colormap = 'Blues', title = conference, subplots = True, legend = False, labels = ['' for x in np.arange(len(df))])
    plt.ylabel('')
    
    plt.legend( list(df.index),  bbox_to_anchor=(1.1, 1),
              fontsize = "xx-small")
    io = StringIO()
    plt.savefig(io, format='png')
    img = base64.encodestring(io.getvalue())
   
    io = StringIO()
    plt.savefig(io, format='png')
    data = base64.encodestring(io.getvalue())
    script = '''<img src="data:image/png;base64,{}";/>'''
    return fig

def getBar(df, conference):
    
    plt.figure()
    ax =  df.plot(kind = 'bar', colormap = 'ocean', title = conference, subplots = True,legend = False)
   
    io = StringIO()
    plt.savefig(io, format='png')
    img = base64.encodestring(io.getvalue())
   
    io = StringIO()
    plt.savefig(io, format='png')
    data = base64.encodestring(io.getvalue())
    script = '''<img src="data:image/png;base64,{}";/>'''
    
def getBar2(df, conference, yaxis, xaxis, ylabel = 'count', xlabel = 'trada'):
    plt.figure()
    fig = sns.barplot(data = df, 
                      y = yaxis, 
                      x = xaxis, 
                      palette='Blues')
    fig.set_ylabel(ylabel)
    fig.set_xlabel(xlabel)
    
    io = StringIO()
    plt.savefig(io, format='png')
    img = base64.encodestring(io.getvalue())
   
    io = StringIO()
    plt.savefig(io, format='png')
    data = base64.encodestring(io.getvalue())
    script = '''<img src="data:image/png;base64,{}";/>'''
    
    return fig

def getHeatMap(data_frame, indexCol = 'confName', cols = 'pubYear', vals = 'counts'):
    plt.cla()
    fig = sns.heatmap(data_frame.pivot_table(index=indexCol, 
                                             columns=cols, 
                                             values=vals),
                                             cmap = 'Blues')
    
    io = StringIO()
    plt.savefig(io, format='png')
    img = base64.encodestring(io.getvalue())
   
    io = StringIO()
    plt.savefig(io, format='png')
    data = base64.encodestring(io.getvalue())
    script = '''<img src="data:image/png;base64,{}";/>'''
    plt.close()
    return fig

def getLine(data_frame, xaxis = 'confName', yaxis = 'counts'):
    plt.cla()
    fig = sns.swarmplot(data = data_frame, x=xaxis, y = yaxis,palette = 'Blues')
    
    io = StringIO()
    plt.savefig(io, format='png')
    img = base64.encodestring(io.getvalue())
   
    io = StringIO()
    plt.savefig(io, format='png')
    data = base64.encodestring(io.getvalue())
    script = '''<img src="data:image/png;base64,{}";/>'''
    plt.close()
    return fig

In [ ]:
def getContentsconf():
    with sqlite3.connect(mydb) as con:
        sqlcmd = "SELECT Conf, Year FROM ABSTRACTSTOTAL"
        df = pd.read_sql_query(sqlcmd, con)
        myt = []
    
        
        conferences = list(df['Conf'].unique())
    
   
        for conf in conferences:
        
            entry = {}
            entry['conf'] = conf
        
            subDF = df.query('Conf == "%s"' % conf).groupby('year').count()
            entry['counts'] = subDF.to_html()
            
            image = getPieOne(subDF, conf)
            entry['Pie']  = image
            
            subDF.reset_index(inplace = True)
            image2 = getBar2(subDF, conf, xaxis = 'year', yaxis = 'Conf')
            entry['Bar'] = image2
        
            myt.append(entry)
    
    return dict(data = myt), df

In [ ]:
g, d = getContentsconf()

In [ ]:
d.reset_index(inplace= True)

In [ ]:
g = getBar2(d, 'EC', xaxis = 'year', yaxis = 'Conf')


In [ ]:

kwdf = keywordcts.sort_values(ascending = False)


new = pd.DataFrame(kwdf.head(10)).rename(columns = {'keyword' : 'count'})
f = getPieOne(new, ('ECSA', 2007) )
new.reset_index(inplace = True)
new
f

In [ ]:
df = d.groupby(['Conf', 'year'])['Conf'].count().reset_index(name="counts")
sns.heatmap(df.pivot_table(index='year', columns='Conf', values='counts'),
                                             cmap = 'Blues')

In [ ]:
sns.heatmap(df.pivot_table(index='year', columns='Conf', values='counts'),
                                             cmap = 'Blues')

In [ ]:
table = []
for idx in cts.index.get_values():
    entry = {}
    kw = cts.iloc[idx].keyword
    entry['kw'] = cts.iloc[idx].keyword
    entry['counts'] = cts.iloc[idx].counts
    mygroup = f.get_group(kw)
    entry['papers'] = list(mygroup.paperID)
    entry['conferences'] = list(mygroup.confName)
    entry['years'] = list(mygroup.pubYear)
    table.append(entry)

In [ ]:
pd.DataFrame(table).head()

In [ ]:
m= getPapersKWgroup2()
m.head()

In [ ]:
m.head()

In [ ]:
def getKWTRends(kw, grouper):
    
    m, f = getPapersKWgroup(grouper)
    
    query2 = '"%s" == keyword' %kw
    
    data_frame = m.copy()
    data_frame.query(query2, inplace = True)
    new = data_frame.copy()
    
    def findKWTrend(df, KWgrouper = ["pubYear", "confName"]):
        labels = {'ECSA' : 0,
                  'QoSA' : 1,
                  'WICSA' : 2}
        df = df.groupby(KWgrouper)['keyword'].count().reset_index(name="counts")
        df['confCode'] = df.confName.apply(lambda name: labels[name])
        try:
            return getHeatMap(df)
        except:
            return 'no data'
        
    
    image = findKWTrend(new)
    
    myentry = [{'table' : data_frame.to_html(),
              'trend'  : image
               }]
    
    return dict(data = myentry)

In [ ]:
t = getKWTRends('Research', 'keyword')
t = getKWTRends('Research', 'keyword')

In [ ]:
new = t.groupby(["pubYear", "confName"])['keyword'].count().reset_index(name="counts")
labels = {'ECSA' : 0,
         'QoSA' : 1,
         'WICSA' : 2}
new['confCode'] = new.confName.apply(lambda name: labels[name])
new.head()

In [ ]:
#sns.heatmap(new.pivot_table( index='confName', columns='pubYear', values='counts'), cmap = 'Blues')
getHeatMap(new)

In [ ]:
ctsmerge.head()

In [ ]:
def getPaperKW():
    m, data_frame = getPapersKWgroup('paperID')
    entries = []
    for each in data_frame.groups:
        entry = {}
        entry['paperID'] = each
        entry['keywords'] = [key for key in data_frame.get_group(each)['keyword']]
        
        entries.append(entry)
    return dict(data = entries)

In [ ]:
j = getPaperKW()

In [ ]:
j

In [ ]:
def search_kw(kw):
    print "kw", kw
    
    
    m, f = getPapersKWgroup('keyword')
    cts = m.groupby(["keyword"])["keyword"].count().reset_index(name="counts")
    ctsmerge = cts.merge(m, on = 'keyword').groupby('keyword')   
    
    try:
        print kw
        subgroup = ctsmerge.get_group(kw)
        mytable = []
            
        for idx in subgroup.index.get_values():
            print idx
            entry = {}
            entry['paperID'] = subgroup.loc[idx].paperID
            entry['Title'] = subgroup.loc[idx]['title']
            entry['Conference'] = subgroup.loc[idx]['confName']   
            entry['PublicationYear'] = subgroup.loc[idx]['pubYear'] 
            mytable.append(entry)
       
        return dict(data = mytable)
    except:
        print (kw,'subgroupfail')
        entry = {'paperID': 'No Keyword Found',
                     'Title': 'No Keyword Found',
                     'Conference': 'No Keyword Found',
                     'PublicationYear': 'No Keyword Found'
                     }
        mytable = [entry]
        return dict(data = mytable)


kw =     'aspectual concept'

In [ ]:
kw2 = 'wireless sensor networks'

In [ ]:
search_kw(u'vulnerability')


# Affiliation_Paper Composite 
## necessary?

In [67]:
paperAffiliationDF = cmd.createAFFILIATIONPAPERTable(paperDF, affilDF)

Opened Abstracts_DB.db database successfully
Records created successfully


In [68]:
paperAffiliationDF.tail()
#paperAffiliationDF.to_csv('Tables_v1/paperAffiliation.csv', sheet_name = 'paperAffil')

,paperID,affilID
1481,1482,1107
1482,1483,379
1483,1484,379
1484,1485,971
1485,1486,412


In [ ]:
paperDF[['paperID', 'pubYear']].merge(paperAffiliationDF, on = 'paperID').tail()

# PAPERAUTHORS COMPOSITE

In [72]:
authorPaper = cmd.createPAPERAUTHORTable(paperDF[['paperID', 'authors']])
#authorPaper.to_csv('Tables_v1/authorPaper.csv', sheet_name = 'paperAuthor')

Opened Abstracts_DB.db database successfully
Records created successfully


In [73]:
authorPaper.head(20)

,paperID,authorName
0,1,"Lopez Herrejon, Roberto E"
1,1,"Egyed, Alexander"
2,2,"Perovich, Daniel"
3,2,"Rossel, Pedro O"
4,2,"Bastarrica, Mar a Cecilia"
5,3,"Perovich, Daniel"
6,3,"Rossel, Pedro O"
7,3,"Bastarrica, Mar a Cecilia"
8,4,"Perovich, Daniel"
9,4,"Rossel, Pedro O"


In [ ]:
paperDF[['paperID', 'title']].merge(authorPaper, on = 'paperID').head(20)

In [ ]:
from flask import *

In [ ]:
confDF.to_html()

In [ ]:
confDF.plot(kind = 'box')

In [ ]:
import numpy as np
def createPAPERAUTHORdf(db = DEFAULTDB):
    ''' Creating the PAPERAUTHOR COMPOSITE Table with 
    
             : param data_frame : Pandas DataFrame from which to create the PAPER TABLE 
                 : - parsed paper table!
             : param db : str. Name of db. (ie. 'Abstracts.db')
                 : default 'test.db'
             : param DFCol : tuple of strings Column names of data_frame with groupby cols.
                 : default = [paperID','terms']
                 : alternatively : enter with already unique table.
             : param table : str. Name of Table to create or insert.
                 : default : 'PAPERAUTHOR'
             : param tableCols : tuple of str. Table keywords as tuple of strings
                 : default  : ('paperID', 'authorName')
             : param foreignKey : tuple of str. Foreign key names:
                 : default : 'PAPER', 'AUTHORS'
         
             : output : Pandas DataFrame as output for inspection
             : output : sql table created with rows inserted.
         '''
    def rech(names):
        names2 = []
        for a in names.split("'"):
            if a.strip() == "," :
                pass
            elif a.strip() == '':
                pass
            else:
                names2.append(a)
        return names2
    
    def createdf(df):
        entries = []
        
        for each in df.index:
            paperID = df.iloc[each].paperID
            authors = df.iloc[each].NEWAUTHORS
            index = np.array([paperID]*len(authors))
            listau = zip(index, authors)
    
            entries.extend(listau)
    
        newdf = pd.DataFrame(entries)
        newdf.rename(columns = {0:'paperID', 1:'authorName'}, inplace = True)
        newdf.authorName = newdf.authorName.apply(lambda au: au.replace(",",' '))
    
        return newdf
    
    with sqlite3.connect(db) as con:
        sqlcmd = "SELECT * FROM AUTHORS"
        
        audf = pd.read_sql_query(sqlcmd, con)
        
        sqlcmd2 = "SELECT paperID, authors FROM PAPER"
        papdf = pd.read_sql_query(sqlcmd2, con)
        
        
        
        papdf['NEWAUTHORS'] = papdf["authors"].apply(lambda a: a.strip("[]'"))
        papdf['NEWAUTHORS'] = papdf["NEWAUTHORS"].apply(lambda a: rech(a))
        
        data_frame = createdf(papdf)
        
        return data_frame
    
def createPAPERAUTHORTable( db = DEFAULTDB, table = "PAPERAUTHOR"):
    with sqlite3.connect(db) as con:
        #create dataframe
        df = createPAPERAUTHORdf()
        
        print df.head()
        cur = con.cursor() 
        #drops the PaperAffilation table if it exist in db already
        cur.execute("DROP TABLE IF EXISTS " + table)
        #and recreates it
        cur.execute("CREATE TABLE " + table + "(\
            paperID INT, \
            authorName TEXT)"
                       );
        
        #insert into the table
        df.to_sql(table, con, flavor='sqlite', 
                        schema=None, if_exists='append', 
                        index=False, index_label=None,
                        chunksize=None, dtype=None)
        print("Records created successfully");
        
        sql = "SELECT * FROM " + table
        testerDF = pd.read_sql_query(sql, con)
        
        return testerDF

In [ ]:
dd = createPAPERAUTHORTable()
dd.head()

# PAPERAUTHOR

In [ ]:
testerDF.dropna

Queries:
As for the given queries: 
papers with key word grouped by year.   
- PaperID ---> KeywordFK, PaperID --> PublicationID:year

author's most buzzy words : 
- authorID ---> paperID -->KeyWordId (count) (sort).max

paper's affiliation grouped by conference and year  ??
- paper ID:affiliation ---> publicationID:year -->ConferenceID:type

conference/year and the keywords in order of frequency (ie word cloud visualization maybe?)  <<_--this might break it?
- confID -->publicationID --> PaperIDs-->KeywordIDs (count) (sort descending)

Affiliation's authors
- authorsID--: papersIds: affiliation  (i need to check if an author can have multiple affiliations, I can't see why not if they change universities or companies after 10 years)


Most frequent affiliations per conference
paperIDs: affiliation --> publicationIDs --> ConferenceID

In [ ]:
q1 = pk.merge(paperDF[['paperID', 'pubYear']], on = 'paperID')


In [ ]:
len(q1['keyword'].unique())

In [ ]:
len(authorPaper)

In [ ]:
authorAffiliation = authorPaper.merge(paperDF[['paperID', 'affiliation', 'pubYear']])

In [ ]:
authorAffiliation.query('"Muccini, Henry" in authorName').sort('pubYear')

In [ ]:
#paper's affiliation grouped by conference and year ??
papafiil = paperAffiliationDF.merge(paperDF[['paperID', 'pubYear', 'confName']], on = 'paperID')

In [ ]:
groupPA = papafiil.groupby(['pubYear', 'confName'])

In [ ]:
groupPA.aggregate('count')

In [ ]:
conf